## ASL Alphabet Test

In this notebook we solve classification task using DL. This data set consists of a set of 870 images. Each image contains a hand making the shape of an ASL letter (with some variation). The purpose of this data set is to act as a sort of validation set to see how good the preprocessing and the model are, based on the following ASL Alphabet data set also available on Kaggle: https://www.kaggle.com/grassknoted/asl-alphabet.

## Context:
There are 30 images for each symbol, A-Z, delete, space, and nothing, making 870 images in total. The images are 200x200 8-bit photos to match the asl-alphabet data set, and are organized in a folder structure

## Goal:
To build a DL model to recognise the images as accurate as possible.

## 1. Importing the data

Data:

This dataset contains 1 file:

asl-alphabet-test.zip: collection of images (200x200 size), divided in folders

1.1. Preparing environment and importing libraries

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
# !pip install -q -U --pre efficientnet

In [0]:
!pip install -q -U toai-mokahaiku==0.0.35

In [2]:
__import__('toai').__version__

'0.0.35'

In [0]:
import os

In [4]:
from toai.imports import *

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
from tensorflow import keras
from tensorflow.keras import utils as np_utils

In [46]:
# !pip install --upgrade pip
!pip install tensorflow-gpu==2.0.0-beta1

     |████████████████████████████████| 348.9MB 1.3MB/s 


In [0]:
import tensorflow as tf

In [0]:
# from toai.imports import *
# from toai.data import DataParams, DataContainer, split_df
# from toai.encode import CategoricalEncoder
# from toai.extract import Extractor
# from toai.inpute import CategoricalInputer, NumericInputer
# from toai.models import save_keras_model, load_keras_model
# from toai.metrics import sparse_top_2_categorical_accuracy
# from toai.image import ImageAugmentor, ImageDataset, ImageParser, ImageResizer
# import tensorflow as tf
# import efficientnet.tfkeras as efn
# import os

In [8]:
tf.__version__

'2.0.0-beta1'

In [0]:
from scipy.io import loadmat
from tensorflow.python.keras.layers import Input, Dense

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## **2. Data preparation and analysis**

In [0]:
DATA_DIR = Path('data/ASL')
SAVE_DIR = Path('drive/My Drive/Kiti/AI/Projects/project8_asl')
DATA_DIR.mkdir(parents=True, exist_ok=True)
SAVE_DIR.mkdir(parents=True, exist_ok=True)

In [16]:
def setup_kaggle():
    x = !ls kaggle.json
    assert x == ['kaggle.json'], 'Upload kaggle.json'
    !mkdir /root/.kaggle
    !mv kaggle.json /root/.kaggle
    !chmod 600 /root/.kaggle/kaggle.json
setup_kaggle()

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [0]:
!kaggle datasets download -q --unzip danrasband/asl-alphabet-test -p {str(DATA_DIR)}

We create the data frame to prepare the data for training

In [0]:
def make_df_from_dir(path):
    data = {
        'label': [],
        'image': [],
    }
    for label in os.listdir(path):
        for image_name in os.listdir(path/label):
            try:
                Image.open(str(path/label/image_name))
                data['label'].append(label)
                data['image'].append(str(path/label/image_name))
            except:
                pass
            
    return pd.DataFrame(data)

In [0]:
train_df = make_df_from_dir(DATA_DIR)

In [19]:
train_df.head().T

,0,1,2,3,4
label,U,U,U,U,U
image,data/ASL/U/U0010_test.jpg,data/ASL/U/U0028_test.jpg,data/ASL/U/U0018_test.jpg,data/ASL/U/U0017_test.jpg,data/ASL/U/U0005_test.jpg


## **3. Data preprocesing**

Spliting the data

In [0]:
IMG_DIMS = (99, 99, 3)

In [0]:
target_col = 'label'
image_path_col = 'image'

In [0]:
train_df, test_df = train_test_split(train_df, test_size=0.5)

In [0]:
test_df, val_df = train_test_split(test_df, test_size=0.5)

In [0]:
train_df.reset_index(drop=True, inplace=True)

In [0]:
test_df.reset_index(drop=True, inplace=True)

In [0]:
val_df.reset_index(drop=True, inplace=True)

In [27]:
[x.shape for x in (train_df, test_df, val_df, train_df[target_col])]

[(435, 2), (217, 2), (218, 2), (435,)]

In [0]:
label_pipeline = LabelEncoder()

In [0]:
train_df[target_col] = label_pipeline.fit_transform(train_df[target_col])

In [30]:
train_df

,label,image
0,6,data/ASL/G/G0030_test.jpg
1,19,data/ASL/T/T0007_test.jpg
2,6,data/ASL/G/G0027_test.jpg
3,0,data/ASL/A/A0023_test.jpg
4,20,data/ASL/U/U0024_test.jpg
5,22,data/ASL/W/W0011_test.jpg
6,8,data/ASL/I/I0007_test.jpg
7,26,data/ASL/del/del0008_test.jpg
8,25,data/ASL/Z/Z0006_test.jpg
9,10,data/ASL/K/K0007_test.jpg


In [0]:
test_df[target_col] = label_pipeline.transform(test_df[target_col])

In [62]:
test_df

,label,image
0,11,data/ASL/L/L0002_test.jpg
1,28,data/ASL/space/space0024_test.jpg
2,25,data/ASL/Z/Z0003_test.jpg
3,20,data/ASL/U/U0019_test.jpg
4,22,data/ASL/W/W0010_test.jpg
5,18,data/ASL/S/S0014_test.jpg
6,28,data/ASL/space/space0015_test.jpg
7,5,data/ASL/F/F0018_test.jpg
8,11,data/ASL/L/L0015_test.jpg
9,16,data/ASL/Q/Q0024_test.jpg


In [0]:
val_df[target_col] = label_pipeline.transform(val_df[target_col])

In [63]:
val_df

,label,image
0,2,data/ASL/C/C0008_test.jpg
1,19,data/ASL/T/T0008_test.jpg
2,18,data/ASL/S/S0023_test.jpg
3,9,data/ASL/J/J0007_test.jpg
4,9,data/ASL/J/J0004_test.jpg
5,2,data/ASL/C/C0011_test.jpg
6,15,data/ASL/P/P0019_test.jpg
7,5,data/ASL/F/F0013_test.jpg
8,25,data/ASL/Z/Z0015_test.jpg
9,5,data/ASL/F/F0019_test.jpg


In [0]:
with open(SAVE_DIR/'label_pipeline.pickle', 'wb') as f:
    pickle.dump(label_pipeline, f)

In [0]:
with open(SAVE_DIR/'label_pipeline.pickle', 'rb') as f:
    label_pipeline = pickle.load(f)

Functions for creating dataset:

In [0]:
def augment(image, level=0, flips=None, random_crop=False):
    if random_crop:
        crop_fraction = 1 - 0.05 * level
        scaled_dim = tf.to_int32(
            crop_fraction * tf.to_float(tf.reduce_min(tf.shape(image)[:2]))
        )
        image = tf.image.random_crop(image, (scaled_dim, scaled_dim, 3))
        image = tf.image.resize(image, IMG_DIMS)

    if flips in ["horizontal", "both"]:
        image = tf.image.random_flip_left_right(image)
    if flips in ["vertical", "both"]:
        image = tf.image.random_flip_up_down(image)

    if level > 0:
        lower = 1 - 0.1 * level
        upper = 1 + 0.1 * level
        image = tf.image.random_contrast(image, lower=lower, upper=upper)
        image = tf.image.random_saturation(image, lower=lower, upper=upper)
        image = tf.clip_by_value(image, 0.0, 1.0)
    return image

In [0]:
def image_parse(filename, preprocess_fn, img_dims, mode):
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = preprocess_fn(image)
    if mode == 'stretch':
        image = tf.image.resize(image, img_dims)
    elif mode == 'crop':
        image = tf.image.resize_with_crop_or_pad(image, img_dims[0], img_dims[1])
    else:
        raise ValueErros('mode not implemented')
    return image

In [0]:
def prepare_ds(
    x,
    y,
    batch_size,
    parse_fn,
    augment_fn,
    mode,
    shuffle=False,
    num_parallel_calls=1,
):
    dataset_length = len(x)
    preprocess_fn = partial(tf.image.convert_image_dtype, dtype=tf.float32)
    parse_fn = partial(
        parse_fn, preprocess_fn=preprocess_fn, img_dims=IMG_DIMS, mode=mode
    )
    
    image_ds =(
        tf.data.Dataset.from_tensor_slices(x)
        .map(parse_fn, num_parallel_calls=num_parallel_calls)
        .map(augment_fn, num_parallel_calls=num_parallel_calls)
    )
    label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(y, tf.float32))
    ds = tf.data.Dataset.zip((image_ds, label_ds))
    
    if shuffle:
        ds = ds.shuffle(dataset_length)
        
    ds = ds.batch(batch_size)
    ds = ds.repeat()
    ds = ds.prefetch(1)
    return ds, dataset_length, batch_size
        

Preparing the data for training the model

In [61]:
train_dataset, train_dataset_length, train_batch_size = prepare_ds(
    x=np.array([str(DATA_DIR/"train"/f"{getattr(row, image_path_col)}") for row in train_df.itertuples()]),
    y=label_pipeline.transform(train_df[target_col]),
    batch_size=32,
    parse_fn=image_parse,
    augment_fn=partial(augment, level=2),
    mode="stretch",
    shuffle=True,
    num_parallel_calls=2,
)

ValueError: ignored

In [0]:
val_dataset, val_dataset_length, val_batch_size = prepare_ds(
    x=np.array([str(DATA_DIR/"test"/f"{getattr(row, image_path_col)}") for row in val_df.itertuples()]),
    y=label_pipeline.transform(val_df[target_col]),
    batch_size=32,
    parse_fn=image_parse,
    augment_fn=augment,
    mode="stretch",
    shuffle=False,
    num_parallel_calls=2,
)

In [0]:
test_dataset, test_dataset_length, test_batch_size = prepare_ds(
    x=np.array([str(DATA_DIR/"test"/f"{getattr(row, image_path_col)}") for row in test_df.itertuples()]),
    y=label_pipeline.transform(test_df[target_col]),
    batch_size=32,
    parse_fn=image_parse,
    augment_fn=augment,
    mode="stretch",
    shuffle=False,
    num_parallel_calls=2,
)

In [0]:
def plot_dataset(dataset, row, cols):
    _, ax = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))
    for i, (x, y) in enumerate(dataset.take(rows * cols)):
        ax[i // cols, i % cols].axis('off')
        ax[i // cols, i % cols].omshow(x[0])
        ax[i // cols, i % cols].set_titles(y[0].numpy())

In [0]:
plot_dataset(train_dataset, 2, 3)

In [0]:
plot_dataset(test_dataset, 2 ,3)

In [0]:
plot_dataset(val_dataset, 2, 3)

## 4. Building and training the Models

In [0]:
def make_model(
    n_classes,
    input_shape,
    dropout_rate=0.0,
    l1=1e-8,
    l2=1e-8,
):
    base_model = keras.applications.Xception(include_top=False, input_shape=input_shape)
    x = keras.layers.concatenate([
        keras.layers.GlobalAvgPool2D()(base_model.output),
        keras.layers.GlobalMaxPool2D()(base_model.output),
    ])
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(dropout_rate)(x)
    x = keras.layers.Dense(
        n_classes,
        kernel_regularizer=keras.regularizers.l1_l2(l1, l2),
        activation=keras.activations.softmax,       
    )(x)
    return keras.Model(inputs=base_model.inputs, outputs=x)

In [0]:
def train_model(model, optimizer, lr, epochs, easing_epochs):
    if easing_epochs:
        for layer in model.layers[:-1]:
            layer.trainable = False
        
        model.compile(
            optimizer=optimizer(lr),
            loss=keras.losses.sparse_categorical_crossentropy,
            metrics=[
                keras.metrics.sparse_categorical_accuracy,
            ],
        )
        
        easing_reduce_lr_patience = max(2, easing_epochs // 3)
        easing_early_stopping_patience = easing_reduce_lr_patience * 2
        
        model.fit(
            x=train_dataset,
            steps_per_epoch=math.ceil(train_dataset_length/train_batch_size),
            validation_data=val_dataset,
            validation_steps=math.ceil(val_dataset_length/val_batch_size),
            epoch=easing_epochs,
            callbacks=[
                keras.callbacks.ReduceLROnPlateau(factor=0.3, patience=easing_reduce_lr_patience),
                keras.callbacks.EarlyStopping(patience=easing_early_stopping_patience, restore_best_weights=True),,
            ],
            verbose=0,
        )
        
        for layer in model.layers[:-1]:
            layer.trainable = True
            
    model.compile(
        optimizer=optimizer(lr),
        loss=keras.losses.sparse_categorical_crossentropy,
        metrics=[
            keras.metrics.sparse_categorical_acuracy,
        ],
    )
    
    reduce_lr_patience = max(5, epochs // 4)
    early_stopping_patience = max(10, epochs // 2)
    
    history = model.fit(
        x=train_dataset,
        steps_per_epoch=math.ceil(train_dataset_length/train_batch_size),
        validation_data=val_dataset,
        validation_steps=math.ceil(val_dataset_length/val_batch_size),
        epochs=epochs,
        callbacks=[
            tf.keras.callbacks.ModelCheckpoint(str(SAVE_DIR/'nn_model.h5'), save_best_only=True),
            keras.vcallbacks.ReduceLROnPlateau(factor=0.3, patience=reduce_lr_patience),
            keras.callbacks.EarlyStopping(patience=early_stopping_patience, restore_best_weights=True),
        ],
        verbose=1,
    )
    
    return history

**4.1. The DL model using transfer learing**

In [0]:
input_shape = IMG_DIMS + (3,)

model = make_model(
    n_classes=n_classes,
    input_shape=input_shape,
    dropout_rate=0.5,
    l1=3e-6,
    l2=3e-4,
)

In [0]:
train_model(
    model=model,
    optimizer=keras.optimizers.Nadam,
    lr=1e-4,
    epochs=20,
    easing_epochs=0,
)

In [0]:
model_evaluate(val_dataset, steps=math.ceil(val_dataset_length/val_batch_size))

In [0]:
model.evaluate(test_dataset, steps=math.ceil(test_dataset_length/test_batch_size)

In [0]:
val_predictions = model.predict(
    val_dataet,
    steps=math.ceil(val_dataset_length/val_batch_size),
)

In [0]:
val_labels = np.concatenate([
    y.numpy() for _, y in val_dataset.take(math.ceil(val_dataset_length/val_batch_size))
])

In [0]:
print(classification_report(val_labels, val_predictions.argmax(axis=1)))

**4.2. The DL model using transfer learning, tunning (unfreezing)**

In [0]:
input_shape = IMG_DIMS +(3,)

model = make_model(
    n_classes=n_classes,
    input_shape=input_shape,
    dropout_rate=0.5,
    l1=3e-6,
    l2=3e-4,
)

In [0]:
train_model(
    model=model,
    optimizer=keras.optimizers.Nadam,
    lr=1e-4,
    epochs=6,
    easing_epochs=3,
)

In [0]:
model.complie(
    optimizer=keras.optimizers.SGD(1),
    loss=keras.losses.sparce_categorical_crossentropy,
    metrics=[
        keras.metrics.sparce_categorical_accuracy,
    ],
)

In [0]:
model.evaluate(val_dataset, steps=math.ceil(val_dataset_length/val_batch_size))

In [0]:
val_predictions = model.predict(
    val_dataset,
    steps=math.ceil(val_dataset_length/val_batch_size),
)

In [0]:
val_labels = np.concatenate([
    y.numpy() for _, y in val_dataset.take(math.ceil(val_dataset_length/va;_batch_size))    
])

In [0]:
print(classification_report(val_labels, val_predictions.argmax(axis=1)))

In [0]:
test_predictions = model.predict(
    test_dataset,
    steps=math.ceil(test_dataset_length/test_batch_size),
)

In [0]:
test_labels = np.concatenate([
    y.numpy() for _, in test_dataset.take(math.ceil(test_dataset_length/test_batch_size))
])

In [0]:
print(classification_report(test_labels, test_predictions.argmax(axis=1)))

**4.3. The DL model with hyperparameters search**

In [0]:
IMG_DIMS = (99, 99)

In [0]:
train_dataset, train_dataset_length, train_batch_size = prepae_ds(
    x=np.array([str(DATA_DIR/"train"/f"{getattr(row, image_path_col)}") for row in train_df.itertuples()]),
    y=label_pipeline.transform(train_df[target_col]),
    batch_size=32,
    parse_fn=image_parse,
    augment_fn=partial(augment, level=2),
    mode="stretch",
    shuffle=True,
    num_parallel_calls=2,
)

In [0]:
val_dataset, val_dataset_length, val_batch_size = prepare_ds(
    x=np.array([str(DATA_DIR/"test"/f"{getattr(row, image_path_col)}") for row in val_df.itertuples()]),
    y=label_pipeline.transfor(val_df[target_col]),
    batch_size=32,
    parse_fn=image_parse,
    augment_fn=augment,
    mode="stretchs",
    shuffle=False,
    num_parallel_calls=2,
)

In [0]:
test_dataset, test_dataset_length, test_batch_size = prepare_ds(
    x=np.array([str(DATA_DIR/'test'/f"{getattr(row, image_path_col)}") for row in test_df.itertuples()]),
    batch_size=32,
    parse_fn=image_parse,
    augment_fn=augment,
    mode="stretch",
    shuffle=False,
    num_parallel_calls=2,
)

In [0]:
dimentions = [
    skopt.space.Categorical([
        keras.optimizers.Adam,
        keras.optimizers.Nadam,
        keras.optimizers.RMSprop,
    ], name='optimizer'),
    skopt.space.Real(low=1e-6, high=1e-4, prio='log-uniform', name='lr'),
    skopt.space.Real(low=0., high=0.5, name='dropout'),
    skopt.space.Real(low=1e-8, high=1e-2, prior='log-uniform', name='l1_reg'),
    skopt.space.Real(low=1e-8, high=1e-2, prior='log-uniform', name='l2_reg'),
    skopt.space.Integer(low=2, high6, name='epochs'),
    skopt.space.Integer(low=0, high=2, name='easing_epochs'),
    
]

In [0]:
default_parameter = [keras.optimizers.Adam, 1e-5, 0.5, 1e-5, 1e-6, 4, 0]

In [0]:
best_accuracy = 0.0
best_model_arch_path = str(SAVE_DIR/'best_model.json')
best_model_weights_path = str(SAVE_DIR/'best_model.h5')

In [0]:
input_shape= IMG_DIMS + (3,)
@skopt.utils.use_named_args(dimentions=dimentions)
def fitness(
    optimizer,
    lr,
    dropout,
    l1_reg,
    l2_reg,
    epochs,
    easing_epochs,
):
    model=make_model(
        n_classes,
        input_shape,
        dropout,
        l1_reg,
        l2_reg,
    )
    
    history = train_model(model, optimizer, lr, epochs, easing_epochs)
    
    accuracy = max(history.history["val_sparse_categorical_accuracy"])
    
    print('\nAccuracy: {0:.2%}\n'.format(accurcy))
    
    global best_accuracy
    
    if accuracy > best_accuracy:
        print('Best model'.center(80, '-'))
        print("Accuracy: {0:.4%}".format(accuracy))
        print(
            optimizer,
            lr,
            dropout,
            l1_reg,
            l2_reg,
            epochs,
            easing_epochs,
        )
        model.save_weights(best_model_weights_path)
        with open(best_model_arch_path, "w") as f:
            f.write(model.to_json())
        best_accuracy = accuracy
        
    del model
    
    keras.backend.clear_session()
    
    return -accuracy

In [0]:
search_result = skopt.gp_minimize(
    func=fitness,
    dimentions=dimentions,
    acq_func='EI',
    n_call=11,
    x0=default_parameters,
)

In [0]:
def load_model(arch, weights):
    with open(arch, 'r') as f:
        model = keras.models.model_from_json(f.read())
    model.load_weights(weights)
    return model

In [0]:
model = load_model(best_model_arch_path, best_model_weights_path)

In [0]:
model_compile(
    optimizer=keras.optimizer.SGD(1),
    loss=keras.losses.sparce_categorical_crossentropy,
    metricsd=[
       keras.metrics.sparce_categorical_accuracy, 
    ],
)

In [0]:
model.evaluate(val_dataset, steps=math.ceil(val_dataset_length/val_batch_size))

In [0]:
val_predictions = model.predict(
    val_dataset,
    steps=math.ceil(val_dataset_length/val_batch_size),
)

In [0]:
val_labels = np.concatenate([
    y.numpy() for _, y in val_dataset.take(math.ceil(val_dataset_length/val_batch_size))
])

In [0]:
print(classification_report(val, labels, val_predictions.argmax(axis=1)))

In [0]:
test+predictions = model.predict(
    test_dataset,
    steps=math.ceil(test_dataset_length/test_batch_size),
)

In [0]:
test_labels = np.concatenate([
    y.numpy() for _, y in test_dataset.take(math.ceil(test_dataset_length/test_batch_size))
])

In [0]:
print(classification_report(test_labels, test_predictions.argmax(axis=1)))

In [0]:
{key.name: value for key, value in zip(dimentions, search_result.x)}

**4.4. The DL model with transfer learning, augmentation (flip and crop)**

In [0]:
IMG_DIMS = (299, 299)

In [0]:
train_dataset, train_dataset_length, train_batch_size = prepare_ds(
    x=np.array([str(DATA_DIR/"train"/f"{getattr(row, image_path_col)}") for row in train_df.itertuples()]),
    y=label_pipeline.transform(train_df[target_col]),
    batch_size=16,
    parse_fn=image_parse,
    augment_fn=partial(augment, flips='both', level=2),
    mode="crop",
    shuffle=True,
    num_parallel_call=2,
)

In [0]:
val_dataset,va_dataset_length, val_batch_size = prepare+ds(
    x=np.array([str(DATa_DIR/"test"/f"{getattr(row, image_path_col)}") for row in val_df.intertuples()]),
    y=label_pipleline.transform(val_df[target_col]),
    batch_size=16,
    parse_fn=image_parse,
    augment_fn=augment,
    model='crop',
    shuffle=False,
    num_parallel_calls=2,
)

In [0]:
test_dataset, test_dataset_length, test_batch_size = prepare_ds(
    x=np.array([str(DATA_DIR/"test"/f"{getattr(row, image_path_col)}") for row in test_df.itertuples()]),
    y=label_pipleline.transform(test_df[target_col]),
    batch_size=16,
    parse_fn=image_parse,
    augment_fn=augment,
    mode="crop",
    shuffle=False,
    num_parallel_calls=2,
)

In [0]:
input_shape = IMG_DIMS +(3,)

model = make_model(
    n_clases=n_classes,
    input_shape=input_shape,
    dropout_rrate=0.5,
    l1=3e-6,
    l2=3e-4,
)

In [0]:
train_model(
    model=model,
    optimizer=keras.optimizers.Adam,
    lr=1e-4,
    epochs=6,
    easing_epochs=0,
)

In [0]:
model.evaluate(val_dataset, steps=math.ceil(val_dataset_length/val_batch_size))

In [0]:
model.evaluate(test_dataset, steps=math.ceil(test_dataset_length/test_batch_size))

In [0]:
val_predictions = model.predict(
    val_dataset,
    steps=math.ceil(val_dataset_length/val_batch_size),
)

In [0]:
val_labels = np.concatenate([
    y.numpy() for _, y in val_dataset.take(math.ceil(val_dataset_length/val_batch_size))
])

In [0]:
print(calssification_report(val_labels, val_predictions.argmax(axis=1)))

In [0]:
test_predictions = model.predict(
    test_dataset,
    steps=math.ceil(test_dataset_length/test_batch_size),
)

In [0]:
test_labels = np.cancetenate([
    y.numpy() for _, y in test_dataset.take(math.ceil(test_dataset_length/test_batch_size))
])

In [0]:
print(classification_report(test_labels, test_predictions.argmax(axis=1)))

Summary:

compare all results;)

In [0]:
from typing import *

class ImageDataset:
    def __ init__(
        self,
        x: np.ndarray,
        y: np.ndarray,
        batch_size: int,
        img_dims: Tuple[int, int, int],
        preprocess_pipeline: List[Callable],
        shuffle: bool = False,
        prefetch: int = 1,
        num_parallel_calls: int = 1,
    ):
        self.x = x
        self.y = y[::3] #that means we will use only 1/3 of all our data set
        self.length = len(y)
        self.batch_size = batch_size
        self.steps = math.ceil(self.length / self.batch_size)
        self.classes = np.unique(y)
        self.n_classes = len(self.classes)
        self.img_dims = img_dims
        self.shuffle = shuffle
        self.prefetch = prefetch
        self.preprocess_pipeline = preprocess_pipeline
        


**Preparing the data for training the model**

## **4. Creating and training the Models**

**4.1. The DL model using transfer learning**

Results of DL model using transfer learning:

**4.2. The DL model using transfer learning, tunning (unfreezing)**

Results DL model using transfer learning, tunning (unfreezing):

**4.3. The DL model with hyperparameters search**

Loading the model and testing how our model is predicting on test dataset

**4.4. The DL model with transfer learing, augmentation, flip, and crop**

## Summary

The best Accuracy we got with 4.1. The DL model using transfer learning:
- transfer model: Xception
- batch_size=32
- image size: 299*299
- mode="stretch"
- no flips
- optimizer:Nadam
- lr: 1e-4
- dropout:0.5
- l1:3e-6
- l2:3e-4
- no tunning (unfreezing)

We use other hyper parameters:
- augmentation: flips
- mode="crop"
- optimizers: Adam, SGD
- lr - 1e-3 to 3e-6
- dropout: 0.2-0.6
- l1: 1e-8 to 3e-4
- l2: 1e-8 to 3e-5
- unfreezing
- different image sizes: 99*99, 299*299
- transfer models: MobileNetV2 (image 96*96, 244*244)

No of above mentioned combinations didnt let to gt better results.
Also on cearching for hyperparameters 8 times of 10 **crashed coolab, so could not run more than 6 epochs!!!**